### Request Predictions

In [166]:
import pandas as pd
import requests
import time

def test(i: int, j: int, print_result: bool = True):
    """
    모델 예측에 대한 테스트용 함수 입니다.
    i,j는 test에 사용할 row 범위를 나타냅니다.
    print_result는 응답 결과에 대한 출력 여부를 설정합니다.
    """

    X_test = pd.read_csv("data/X_test.csv").drop(columns=["Name", "group"])
    y_test = pd.read_csv("data/y_test.csv")

    # request Prediction
    payloads = X_test.iloc[i:j].to_dict('list')
    req = requests.post("http://localhost:8000/predict", json=payloads)

    # to_json
    req_json = req.json()
    if print_result:
        from pprint import pprint

        pprint(req_json)

    # response Customer_Adaptivity_Level
    if req.status_code == 200:
        customer_id: list = req_json["data"]["table_id"]
        customer_Transported: list = y_test[i:j].T.values.tolist()[0]

        payloads_result = []
        for id, level in zip(customer_id, customer_Transported):
            result = dict(id=id, Transported=level)
            payloads_result.append(result)
        time.sleep(0.1) # predict의 background task 수행을 기다리기 위함.
        req = requests.post("http://localhost:8000/result", json=payloads_result)
    else:
        raise ConnectionError(f'current status code is "{req.status_code}" ')

    return None


if __name__ == "__main__":
    from random import sample

    # 242 = len(y_test)
    a, b = sorted(sample(range(242), 2))
    # a = 1
    # b = 5

    print(f"요청 rows 개수 : {b-a}")
    print("요청 결과를 확인하고 싶은 경우 `print_result`를 True로 설정해주세요.")
    test(a, b, print_result=False)

요청 rows 개수 : 80
요청 결과를 확인하고 싶은 경우 `print_result`를 True로 설정해주세요.


In [171]:
from sklearn.metrics import *
import matplotlib.pyplot as plt
import datapane as dp

def create_metrics(y_true, y_pred, average="binary"):

    # create confusion_matrix plots
    num = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[True, False])
    per = confusion_matrix(y_true=y_true, y_pred=y_pred, normalize="pred", labels=[True, False])

    num_con_mat = ConfusionMatrixDisplay(confusion_matrix=num, display_labels=[True, False])
    per_con_mat = ConfusionMatrixDisplay(confusion_matrix=per, display_labels=[True, False])

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 8))

    num_con_mat.plot(cmap=plt.cm.binary, ax=axes[0], colorbar=False)
    per_con_mat.plot(cmap=plt.cm.Blues, ax=axes[1], colorbar=False)

    axes[0].set_title("Counts")
    axes[1].set_title("Rate")
    plt.tight_layout()
    fig.savefig('hi.png',bbox_inches='tight')
    plt.close()

    return {
        "plot": fig,
        "precision": precision_score(
            y_true,
            y_pred,
            average=average,
        )
        * 100,
        "recall": recall_score(y_true, y_pred, average=average) * 100,
        "accuracy_score": accuracy_score(y_true, y_pred) * 100,
    }

import joblib 
X_test = pd.read_csv("data/X_test.csv").drop(columns=["Name", "group"])
rf = joblib.load('models/random_forest.pkl')
y_pred = rf.predict(X_test)
y_test = pd.read_csv("data/y_test.csv")

create_metrics(y_test,y_pred)

{'plot': <Figure size 576x576 with 2 Axes>,
 'precision': 82.37082066869301,
 'recall': 81.01644245142003,
 'accuracy_score': 81.61875945537065}